In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Data Point

In [2]:
from recfldgrn.datapoint import PatientDP, RANGE_SIZE

PID = 'P4'
rec_folder = 'data/ProcData/RecFolder/'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
Pat

In [3]:
RecName = 'P'
df = Pat.get_df_rec(RecName)
df

,PID,age,basicInfo
0,P4,23,Male&A


# Task Selection

In [4]:
TaskName = 'Task2YearXXX'

ELIG_PAT_SET = [f'P{i}' for i in range(0, 8)]
ELIG_PAT_SET

['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']

In [5]:
import os
import pandas as pd

Tensor_folder = 'data/ProcData/TensorFolder'
TaskTensor_folder = os.path.join(Tensor_folder, TaskName)

if not os.path.exists(TaskTensor_folder): os.makedirs(TaskTensor_folder)

print(TaskTensor_folder)

data/ProcData/TensorFolder/Task2YearXXX


In [6]:
RecName = 'P'

df = pd.DataFrame()
for PID in ELIG_PAT_SET:
    Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
    dfx = Pat.get_df_rec(RecName)
    # make some operation to select good dfx
    dfx = dfx 
    df = pd.concat([df, dfx])
    
print(df.shape)
df

(8, 3)


,PID,age,basicInfo
0,P0,52,Male&B
0,P1,21,Male&B
0,P2,23,Female&B
0,P3,77,Female&B
0,P4,23,Male&A
0,P5,73,Female&D
0,P6,34,Female&A
0,P7,76,Male&C


# Load Meta Info

In [7]:
import os
import pandas as pd

fldgrn_folder = 'data/ProcData/FldGrnInfo'

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')

df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,P,[],[PID],"[basicInfo, age]",basicInfo,basicInfoDftGrn,basicInfo-basicInfoDftGrn,P@basicInfo-basicInfoDftGrn,"{'idx2v': {0: '_padding', 1: 'Male', 2: 'B', 3..."
1,P,[],[PID],"[basicInfo, age]",age,AgeNumeDftGrn,age-AgeNumeDftGrn,P@age-AgeNumeDftGrn,"{'v2idx': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."


# RecFldGrn `P@basicInfo-basicInfoDftGrn`

In [8]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info
from recfldgrn.utils import get_compressed_df
from recfldgrn.datapoint import write_df_to_folders

In [9]:
from recfldgrn.utils import get_highorder_input_idx

# def get_highorder_input_idx(df, recfield2grain, 
#                             v2idx, prefix_ids, focal_ids, 
#                             Field2Grain_2_get_GrnIdxList_Fn, 
#                             Field2Grain_2_get_GrnWgtList_Fn):
    
#     RecName, field = recfield2grain.split('-')[0].split('@')
#     grain = recfield2grain.split('-')[-1]
    
#     suffix = '_idx' if 'Nume' not in recfield2grain else '_wgt'
#     if suffix == '_idx':
#         get_grn_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
#         df[recfield2grain + suffix] = df[field].apply(lambda x: get_grn_idx_fn(x, v2idx))
#     elif suffix == '_wgt':
#         get_grn_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
#         df[recfield2grain + suffix] = df[field].apply(lambda x: get_grn_wgt_fn(x))
#     else:
#         raise ValueError(f'Incorrect suffix "{suffix}"')
    
    
#     full_recfldgrn_name_c = recfield2grain + suffix
#     top_id = df.columns[0] # need to double check this.
    
#     if len(prefix_ids) > 0:
#         prefix_ids_c = prefix_ids
#         df_Rec = df[prefix_ids_c + [full_recfldgrn_name_c]]
#         for i in range(len(prefix_ids_c)):
#             df_Rec, full_recfldgrn_name_c, prefix_ids_c = get_compressed_df(df_Rec, full_recfldgrn_name_c, prefix_ids_c)
#     else:
#         df_Rec = df[focal_ids + [full_recfldgrn_name_c]]

#     # df_p = df_Rec#.set_index(top_id)
#     df_p = df_Rec.reset_index(drop = True)
#     return df_p

In [10]:
TaskTensor_folder

'data/ProcData/TensorFolder/Task2YearXXX'

In [11]:
recfield2grain = 'P@basicInfo-basicInfoDftGrn'

d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)



fullTaskTensor_folder = os.path.join(TaskTensor_folder, recfield2grain)
print(df.shape, fullTaskTensor_folder)
if not os.path.exists(fullTaskTensor_folder): os.makedirs(fullTaskTensor_folder)
write_df_to_folders(recfield2grain, fullTaskTensor_folder, df_p)

df_p

(8, 4) data/ProcData/TensorFolder/Task2YearXXX/P@basicInfo-basicInfoDftGrn


,PID,P@basicInfo-basicInfoDftGrn_idx
0,P0,"[1, 2]"
1,P1,"[1, 2]"
2,P2,"[3, 2]"
3,P3,"[3, 2]"
4,P4,"[1, 4]"
5,P5,"[3, 5]"
6,P6,"[3, 4]"
7,P7,"[1, 6]"


# RecFldGrn `P@age-AgeNumeDftGrn`

In [12]:
recfield2grain = 'P@age-AgeNumeDftGrn'


d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)


fullTaskTensor_folder = os.path.join(TaskTensor_folder, recfield2grain)
print(df.shape, fullTaskTensor_folder)
if not os.path.exists(fullTaskTensor_folder): os.makedirs(fullTaskTensor_folder)
write_df_to_folders(recfield2grain, fullTaskTensor_folder, df_p)

df_p

(8, 5) data/ProcData/TensorFolder/Task2YearXXX/P@age-AgeNumeDftGrn


,PID,P@age-AgeNumeDftGrn_wgt
0,P0,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
1,P1,"[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0...."
2,P2,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0...."
3,P3,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
4,P4,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0...."
5,P5,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
6,P6,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0...."
7,P7,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
